In [30]:
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
pc.settings.set_subscription_key('85c168c81886441d89c30d0bd8613cc0')

In [31]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from tqdm import tqdm
scl_colormap = np.array(
    [
        [252,  40, 228, 255],  # 0  - NODATA - MAGENTA
        [255,   0,   4, 255],  # 1  - Saturated or Defective - RED
        [0  ,   0,   0, 255],  # 2  - Dark Areas - BLACK
        [97 ,  97,  97, 255],  # 3  - Cloud Shadow - DARK GREY
        [3  , 139,  80, 255],  # 4  - Vegetation - GREEN
        [192, 132,  12, 255],  # 5  - Bare Ground - BROWN
        [21 , 103, 141, 255],  # 6  - Water - BLUE
        [117,   0,  27, 255],  # 7  - Unclassified - MAROON
        [208, 208, 208, 255],  # 8  - Cloud - LIGHT GREY
        [244, 244, 244, 255],  # 9  - Definitely Cloud - WHITE
        [195, 231, 240, 255],  # 10 - Thin Cloud - LIGHT BLUE
        [222, 157, 204, 255],  # 11 - Snow or Ice - PINK
    ],
    dtype="uint8",
)
resolution = 20 # meters per pixel
scale = resolution / 111320.0 # degrees per pixel for CRS:4326 

In [32]:
# Create a mask for no data, saturated data, clouds, cloud shadows, and water
def create_cloud_mask(xx):
    cloud_mask = \
        (xx.SCL != 0) & \
        (xx.SCL != 1) & \
        (xx.SCL != 3) & \
        (xx.SCL != 6) & \
        (xx.SCL != 8) & \
        (xx.SCL != 9) & \
        (xx.SCL != 10) 
    return cloud_mask
def further_remove_cloud(items):
    i = 0
    removal_list = []
    while i < len(items):
        if items[i].properties['eo:cloud_cover'] > 60:
            removal_list.append(i)
        i += 1
    k = 0
    for k in range(len(removal_list)):
        items.pop(removal_list[k]-k)
    return items

In [33]:
crop_presence_data = pd.read_csv("Crop_Location_Data.csv")
box_size_deg = 0.00898 # Surrounding box in degrees, 5x5 pixels, 20m per pixel
def get_ndvi_data(latlong, time):
    latlong=latlong.replace('(','').replace(')','').replace(' ','').split(',')
    min_lat = float(latlong[0]) - box_size_deg/2
    min_long = float(latlong[1]) - box_size_deg/2
    max_lat = float(latlong[0]) + box_size_deg/2
    max_long = float(latlong[1]) + box_size_deg/2
    bbox_of_interest = (min_long, min_lat, max_long, max_lat)
    time_of_interest = time
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-2-l2a"], bbox=bbox_of_interest, datetime=time_of_interest)
    items = list(search.get_all_items())
    items = further_remove_cloud(items)
    xx = stac_load(
        items,
        bands=["red", "green", "blue", "nir", "SCL"],
        crs="EPSG:4326", # Latitude-Longitude
        resolution=scale, # Degrees
        chunks={"x": 2048, "y": 2048},
        dtype="uint16",
        patch_url=pc.sign,
        bbox=bbox_of_interest
    )
    cleaned_data = xx.where(create_cloud_mask(xx)).astype("uint16")
    mean_clean = cleaned_data.mean(dim=['longitude','latitude']).compute()
    ndvi_mean_clean = (mean_clean.nir-mean_clean.red)/(mean_clean.nir+mean_clean.red)
    ndvi_mean_clean=ndvi_mean_clean.to_numpy()
    ndvi_mean_clean = ndvi_mean_clean[~np.isnan(ndvi_mean_clean)]
    try:
        average = sum(ndvi_mean_clean)/len(ndvi_mean_clean)
        return max(ndvi_mean_clean), min(ndvi_mean_clean), average, np.std(ndvi_mean_clean)
    except ValueError:
        return 0,0,0,0

In [34]:
time = "2021-12-01/2022-06-01"
max_ndvis = []
min_ndvis = []
avg = []
std = []
#max_ndvi, min_ndvi = get_ndvi_data(crop_presence_data['Latitude and Longitude'].iloc[400], time)
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    max_ndvi, min_ndvi, single_avg, single_std = get_ndvi_data(coordinates, time)
    max_ndvis.append(max_ndvi)
    min_ndvis.append(min_ndvi)
    avg.append(single_avg)
    std.append(single_std)
    

crop_presence_data['max_ndvi'] = max_ndvis
crop_presence_data['min_ndvi'] = min_ndvis
crop_presence_data['avg'] = avg
crop_presence_data['std'] = std
crop_presence_data.to_csv('DT/max_min_crop_data.csv', index=False)

100%|██████████| 600/600 [28:26<00:00,  2.84s/it]


In [35]:
crop_data = pd.read_csv('DT/max_min_crop_data.csv')
target = []
for i in crop_data.index:
    if crop_data['Class of Land'].iloc[i] == "Rice":
        target.append(1)
    else:
        target.append(0)
crop_data['target'] = target
crop_data

,Latitude and Longitude,Class of Land,max_ndvi,min_ndvi,avg,std,target
0,"(10.323727047081501, 105.2516346045924)",Rice,0.829053,0.062270,0.346829,0.205330,1
1,"(10.322364360592521, 105.27843410554115)",Rice,0.833382,-0.042734,0.337155,0.229924,1
2,"(10.321455902933202, 105.25254306225168)",Rice,0.831319,0.053462,0.350250,0.204107,1
3,"(10.324181275911162, 105.25118037576274)",Rice,0.830906,0.054959,0.343384,0.209116,1
4,"(10.324635504740822, 105.27389181724476)",Rice,0.826148,0.023583,0.361130,0.212555,1
...,...,...,...,...,...,...,...
595,"(10.013942985253381, 105.67361318732796)",Non Rice,0.824462,0.323611,0.548386,0.127470,0
596,"(10.01348875642372, 105.67361318732796)",Non Rice,0.820659,0.063777,0.528311,0.153976,0
597,"(10.013034527594062, 105.67361318732796)",Non Rice,0.817224,0.063777,0.525703,0.155322,0
598,"(10.012580298764401, 105.67361318732796)",Non Rice,0.817845,0.063777,0.525824,0.155126,0


In [46]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
X = crop_data[['max_ndvi', 'min_ndvi', 'avg', 'std']]
y = crop_data[['target']]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
dtc = DecisionTreeClassifier()
dtc = dtc.fit(X_train, y_train)

In [47]:
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

text_representation = tree.export_text(dtc)
print(text_representation)
print(dtc.feature_names_in_)
predictions = dtc.predict(X_test)
ac = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
print(ac, f1)

|--- feature_3 <= 0.18
|   |--- feature_3 <= 0.17
|   |   |--- feature_3 <= 0.16
|   |   |   |--- class: 0
|   |   |--- feature_3 >  0.16
|   |   |   |--- feature_3 <= 0.16
|   |   |   |   |--- class: 1
|   |   |   |--- feature_3 >  0.16
|   |   |   |   |--- feature_2 <= 0.41
|   |   |   |   |   |--- feature_0 <= 0.64
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- feature_0 >  0.64
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_2 >  0.41
|   |   |   |   |   |--- class: 0
|   |--- feature_3 >  0.17
|   |   |--- feature_1 <= -0.01
|   |   |   |--- class: 0
|   |   |--- feature_1 >  -0.01
|   |   |   |--- class: 1
|--- feature_3 >  0.18
|   |--- feature_2 <= 0.29
|   |   |--- class: 0
|   |--- feature_2 >  0.29
|   |   |--- class: 1

['max_ndvi' 'min_ndvi' 'avg' 'std']
1.0 1.0


In [38]:
test = pd.read_csv("test.csv")
result = []
time = "2021-12-01/2022-06-01"
max_ndvis = []
min_ndvis = []
test_avg = []
test_std = []
#max_ndvi, min_ndvi = get_ndvi_data(crop_presence_data['Latitude and Longitude'].iloc[400], time)
for coordinates in tqdm(test['id']):
    max_ndvi, min_ndvi, single_avg, single_std = get_ndvi_data(coordinates, time)
    max_ndvis.append(max_ndvi)
    min_ndvis.append(min_ndvi)
    test_avg.append(single_avg)
    test_std.append(single_std)

test['max_ndvi'] = max_ndvis
test['min_ndvi'] = min_ndvis
test['avg'] = test_avg
test['std'] = test_std
test.to_csv('DT/test_data.csv', index=False)

100%|██████████| 250/250 [14:39<00:00,  3.52s/it]


In [48]:
test_data = pd.read_csv('DT/test_data.csv')
x = test_data[['max_ndvi', 'min_ndvi', 'avg', 'std']]
print(x)
test_pred = dtc.predict(x)
print(test_pred)
target = []
for i in range(len(test_pred)):
    if test_pred[i] == 1:
        target.append("Rice")
    else:
        target.append("Non Rice")
test_data['target'] = target
test_data.drop(columns = ['max_ndvi', 'min_ndvi', 'avg', 'std'], inplace=True)
test_data.to_csv("DT/submissionoriginal.csv", index=False)

     max_ndvi  min_ndvi       avg       std
0    0.890565  0.043304  0.359775  0.204545
1    0.775325 -0.040445  0.370187  0.224815
2    0.816785  0.090762  0.405190  0.184038
3    0.403945 -0.176341  0.260716  0.112069
4    0.927154  0.052555  0.453548  0.258038
..        ...       ...       ...       ...
245  0.166399 -0.055596  0.084468  0.059561
246  0.390869 -0.169374  0.253233  0.112360
247  0.359871 -0.109992  0.224766  0.100547
248  0.794916  0.114287  0.393408  0.182262
249  0.569844 -0.001977  0.342557  0.167900

[250 rows x 4 columns]
[1 1 1 0 1 0 0 1 0 0 0 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0
 0 0 1 1 0 0 1 0 1 1 0 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 0 0 1 1 0 1 0 0 1
 0 0 1 1 0 1 1 1 0 1 1 1 0 1 0 1 0 0 1 0 1 1 1 0 1 1 0 0 0 0 1 0 0 0 0 1 0
 0 0 1 1 0 0 1 0 0 1 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 1
 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 1 1
 0 0 1 0 0 1 1 0 1 1 0 1 1 1 0 0 0 0 1 1 0 0 0 1 1 0 1 1 1 1 0 0 1 0 1 0 